# PROGETTO DI FOUNDATIONS OF COMPUTER SCIENCE

**- Nome:** Davide<br>
**- Cognome:** Sangalli<br>
**- Matricola:** 848013<br>

In [1]:
import pandas as pd
import numpy as np
import datetime
import random
import time

Viene innanzitutto creata una funzione per leggere tutti i dataset necessari e successivamente vengono caricati in memoria

In [2]:
def load_dataset(name):
    df= pd.read_csv(r'{}'.format(name))
    return df

In [3]:
loans_lenders=load_dataset("./FCS/loans_lenders.csv")
loans=load_dataset("./FCS/loans.csv")
lenders=load_dataset("./FCS/lenders.csv")
country_stats=load_dataset("./FCS/country_stats.csv")

### 1) Normalize the loan_lenders table.
In the normalized table, each row must have one loan_id and one lender.

Per realizzare la normalizzazione viene settato l'attibuto *loan_id* come indice e viene splittato l'attributo *lenders*; successivamente vengono duplicati gli indici tramite la funzione **explode()**.

In [4]:
loans_lenders_normalized = (loans_lenders.set_index(['loan_id'])
                   .apply(lambda x: x.str.split(', ').explode())
                   .reset_index())
 
loans_lenders_normalized

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


### 2) For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time.

If any of those two dates is missing, also the duration must be missing.

Dopo aver trasformato gli attributi *disburse time* e *planned expiration time* in formato **datetime**, si è eseguita la differenza tra le due quantità, creando una nuova colonna (*duration*); successivamente si è controllato quanti valori di *duration* fossero nulli.

In [5]:
def get_duration():
    
    loans[['disburse_time', 'planned_expiration_time']]=loans[['disburse_time', 
                                                    'planned_expiration_time']].apply(pd.to_datetime).apply(lambda x: x.dt.date)
    
    loans['duration']=(loans['planned_expiration_time']-loans['disburse_time']).dt.days
    
    print(loans['duration'])
    
    print(loans['duration'].isnull().value_counts())

In [6]:
get_duration()

0          54.0
1          96.0
2          37.0
3          35.0
4          58.0
           ... 
1419602    40.0
1419603    39.0
1419604    51.0
1419605    63.0
1419606    61.0
Name: duration, Length: 1419607, dtype: float64
False    1044962
True      374645
Name: duration, dtype: int64


### 3) Find the lenders that have funded at least twice.

Viene eseguito un merge tra i prestiti che hanno uno stato *funded* e la tabella *loans_lenders_normalized* creata al punto 1).<br>Successivamente vengono restituiti solo i *lenders* che hanno investito almeno due volte. 

In [7]:
def find_loans_funded():
    
    loans_funded = pd.merge(loans['loan_id'].loc[loans['status'] == 'funded'], loans_lenders_normalized, 
                        on = 'loan_id')
    
    print(loans_funded['lenders'].value_counts().loc[lambda x: x>=2])

In [8]:
find_loans_funded()

gooddogg1           167512
trolltech4460       141268
gmct                123164
nms                  96713
themissionbeltco     74067
                     ...  
juan5753                 2
terry9263                2
sonia5902                2
deb3855                  2
melody1952               2
Name: lenders, Length: 893981, dtype: int64


### 4) For each country, compute how many loans have involved that country as borrowers.

Per svolgere questo punto, si calcolano le occorrenze di ogni Paese all'interno del dataset *loans*.

In [9]:
loans['country_name'].value_counts()

Philippines         285336
Kenya               143699
Peru                 86000
Cambodia             79701
El Salvador          64037
                     ...  
Papua New Guinea         1
Canada                   1
Botswana                 1
Mauritania               1
Uruguay                  1
Name: country_name, Length: 96, dtype: int64

### 5) For each country, compute the overall amount of money borrowed.

Per calcolare l'importo complessivo del denaro preso in prestito per ogni Paese, si esegue un raggruppamento per Paese (attributo *country_name*) e si esegue una somma dell'importo del prestito.

In [10]:
country_borrowed = loans.groupby('country_name')['loan_amount'].sum()

country_borrowed

country_name
Afghanistan        1967950.0
Albania            4307350.0
Armenia           22950475.0
Azerbaijan        14784625.0
Belize              150175.0
                     ...    
Vietnam           24681100.0
Virgin Islands       10000.0
Yemen              3444000.0
Zambia             1978975.0
Zimbabwe           5851875.0
Name: loan_amount, Length: 96, dtype: float64

### 6) Like the previous point, but expressed as a percentage of the overall amount lent.

Si considera ora l'importo prestato (attributo *funded_amount*); successivamente si calcola la percentuale di denaro prestato sul denaro preso in prestito

In [11]:
country_lent=loans.groupby('country_name')['funded_amount'].sum()

(country_borrowed/country_lent)*100

country_name
Afghanistan       100.167969
Albania           107.886712
Armenia           113.141941
Azerbaijan        104.180540
Belize            100.000000
                     ...    
Vietnam           105.187880
Virgin Islands           inf
Yemen             103.113001
Zambia            103.435254
Zimbabwe          100.572737
Length: 96, dtype: float64

### 7) Like the three previous points, but split for each year (with respect to "disburse time").

Viene innanzittutto creata una colonna apposita nel dataframe *loans*, chiamata **disburse_year**, ottenuta splittando la colonna **disburse_time** e tenendo solo l'anno.

Successivamente, verranno effettuate le tre operazioni precedenti, ma eseguendo una *group_by* anche  sull'anno, oltre che sul nome del Paese

In [12]:
loans['disburse_year']=loans['disburse_time'].apply(lambda x:x.year)

#### 7.1) For each country and year, compute how many loans have involved that country as borrowers.

In [13]:
loans.groupby(['country_name', 'disburse_year']).size()

country_name  disburse_year
Afghanistan   2007.0            408
              2008.0            370
              2009.0            678
              2010.0            632
              2011.0            247
                               ... 
Zimbabwe      2013.0            426
              2014.0           2078
              2015.0            600
              2016.0            808
              2017.0           1079
Length: 748, dtype: int64

#### 7.2) For each country and year, compute the overall amount of money borrowed.

In [14]:
loans.groupby(['country_name', 'disburse_year'])['loan_amount'].sum()

country_name  disburse_year
Afghanistan   2007.0            194975.0
              2008.0            365375.0
              2009.0            585125.0
              2010.0            563350.0
              2011.0            245125.0
                                 ...    
Zimbabwe      2013.0            678525.0
              2014.0           1311575.0
              2015.0            723625.0
              2016.0            788600.0
              2017.0           1237600.0
Name: loan_amount, Length: 748, dtype: float64

#### 7.3) For each country and year, do like the previous point, but expressed as a percentage of the overall amount lent.

In [15]:
(loans.groupby(['country_name', 'disburse_year'])['loan_amount'].sum() / loans['funded_amount'].sum()) * 100

country_name  disburse_year
Afghanistan   2007.0           0.017252
              2008.0           0.032329
              2009.0           0.051772
              2010.0           0.049846
              2011.0           0.021689
                                 ...   
Zimbabwe      2013.0           0.060037
              2014.0           0.116050
              2015.0           0.064027
              2016.0           0.069776
              2017.0           0.109504
Name: loan_amount, Length: 748, dtype: float64

### 8) For each lender, compute the overall amount of money lent.
For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

Viene creato un dataframe unendo i dataframe *loans_lenders_normalized* e *loans*, successivamente viene creata una nuova colonna, chiamata *average_amount*, che esprime il contributo di ogni ogni membro (assumendo che ad ogni lender corrisponda la stessa somma di denaro). Infine, si raggruppa per *lenders* e si calcola la somma di denaro prestata.

In [16]:
money_lent_lenders = pd.merge(loans_lenders_normalized, loans[['loan_id', 'funded_amount', 'num_lenders_total']], 
                              on = 'loan_id')
    
money_lent_lenders['average_amount'] = money_lent_lenders['funded_amount'] / money_lent_lenders['num_lenders_total']

money_lent_lenders.groupby('lenders')['average_amount'].sum().reset_index(name = 'lent')

,lenders,lent
0,000,1512.095370
1,00000,1249.127691
2,0002,2201.180463
3,00mike00,38.461538
4,0101craign0101,2424.088932
...,...,...
1383794,zzmcfate,57535.947017
1383795,zzpaghetti9994,44.247788
1383796,zzrvmf8538,480.818179
1383797,zzzsai,236.888048


### 9) For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed
Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

Prima di tutto si controlla il numero di *country_code* nulli per i dataframes *loans* e *lenders*

In [17]:
print(loans['country_code'].isna().value_counts())
print(lenders['country_code'].isna().value_counts())

False    1419598
True           9
Name: country_code, dtype: int64
True     1458635
False     890539
Name: country_code, dtype: int64


Come secondo step, si trova la distribuzione per i *country_code*, definendo un metodo di sostituzione per i *country_code* nulli. In particolare, per la sosituzione di valori nulli, si usa una probabilità data dal rapporto tra il numero di *country_code* non nulli e la lunghezza degli stessi (funzione **get_country_code_distibution**). Infine si provvede alla sostituzione di questi (funzione **replace_null_country_codes**). 

In [18]:
def get_country_code_distribution(series):
    
    replaces=[]
    
    not_null_country_codes=pd.unique(series.dropna())
    replaces.append(not_null_country_codes)
    
    replace_probs=series.dropna().value_counts()/len(series.dropna())
    replaces.append(replace_probs)
    
    null_country_code_idxs=series.index[series.isnull()]
    replaces.append(null_country_code_idxs)
    
    replaces.append(len(null_country_code_idxs))
    
    return replaces

In [19]:
def replace_null_country_codes(df,series):
    
    fill_null_country_codes=get_country_code_distribution(series)
    
    country_code_replaces=pd.Series(np.random.choice(fill_null_country_codes[0],p=fill_null_country_codes[1],
                                                size=fill_null_country_codes[3]),index=fill_null_country_codes[2])
    
    if(df=="lenders"):
        
        lenders['country_code'].fillna(country_code_replaces,inplace=True)
        return lenders
    
    elif(df=="loans"):
        
        loans['country_code'].fillna(country_code_replaces,inplace=True)
        return loans

In [20]:
lenders=replace_null_country_codes("lenders",lenders['country_code'])
loans=replace_null_country_codes("loans",loans['country_code'])

Si controlla ora il numero di *country_code* nulli e si verifica che non ce ne sono più dopo aver operato la sostituzione

In [21]:
print(lenders['country_code'].isna().value_counts())
print(loans['country_code'].isna().value_counts())   

False    2349174
Name: country_code, dtype: int64
False    1419607
Name: country_code, dtype: int64


Si trovano ora le nuove distribuzioni per i soldi presi in prestito e per i soldi prestati 

In [22]:
def get_distribution_money_borrowed(df):
    
    new_df_grouped= df.groupby('country_code')['loan_amount'].sum().reset_index(name = 'borrowed')
    
    return new_df_grouped

In [23]:
def get_distribution_money_lent(df_1,df_2):
    
    new_df = pd.merge(df_1, df_2,left_on = 'lenders', right_on = 'permanent_name')
    
    new_df_grouped = new_df.groupby('country_code')['average_amount'].sum().reset_index(name = 'lent')
    
    return new_df_grouped

In [24]:
money_borrowed_by_country=get_distribution_money_borrowed(loans)

In [25]:
money_lent_per_country=get_distribution_money_lent(money_lent_lenders,lenders[['permanent_name', 'country_code']])

Infine si crea un nuovo dataframe, dato dal merge tra i soldi prestati per ogni Paese e i soldi presi in prestito per ogni Paese.<br> Si riempiono con zero i valori nulli all'interno di quest'ultimo dataframe e si crea una nuova colonna (*difference*), data dalla sottrazione tra soldi prestati per ogni Paese e soldi presi in prestito per ogni Paese.

In [26]:
money_lent_borrowed_by_country = pd.merge(money_lent_per_country, money_borrowed_by_country, on = 'country_code', 
                                    how = 'outer')

money_lent_borrowed_by_country.fillna(0, inplace = True)

money_lent_borrowed_by_country['difference'] = money_lent_borrowed_by_country['lent'] - \
                                               money_lent_borrowed_by_country['borrowed']

In [27]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [28]:
money_lent_borrowed_by_country

,country_code,lent,borrowed,difference
0,AD,14587.76532,0.00000,14587.76532
1,AE,1664676.36244,0.00000,1664676.36244
2,AF,96670.50779,1967950.00000,-1871279.49221
3,AG,1428.39955,0.00000,1428.39955
4,AI,1000.84298,0.00000,1000.84298
...,...,...,...,...
230,ZA,489807.03514,1006525.00000,-516717.96486
231,ZM,36042.19179,1978975.00000,-1942932.80821
232,ZW,8995.80754,5851875.00000,-5842879.19246
233,GZ,0.00000,5000.00000,-5000.00000


### 10) Which country has the highest ratio between the difference computed at the previous point and the population?

La funzione **get_ratio** serve per calcolare le richieste del punto 10 e del punto 11.<br>
Il dataframe che contiene le informazioni riguardanti i Paesi è *country_stats*.<br> Per quanto rigurda il punto 10, questo dataframe viene unito al dataframe costruito precedentemente (*money_lent_borrowed_by_country*) e viene restituito il Paese che ha la ratio più alta.<br> Per quanto riguarda il punto 11, quando viene effettuato il merge si prende anche in considerazione l'attributo *population_below_poverty_line*.<br> Grazie al parametro booleano (impostato di default a **False**) è possibile decidere se includere quest'ultimo attributo o meno. 

In [29]:
def get_ratio(dataframe,poverty_line=False):
    
    if(poverty_line):
        
        dataframe=pd.merge(dataframe, country_stats[['country_code', 'population', 
                                    'population_below_poverty_line']], 
                                     on = 'country_code')
        
        dataframe['population_above_poverty_line'] = (dataframe['population'] * 
                                           (100-dataframe['population_below_poverty_line'])) / 100
        
        dataframe=dataframe.set_index('country_code')
        
        print((dataframe['difference'] / dataframe['population_above_poverty_line']).idxmax())
    
    else:
        
        dataframe=pd.merge(dataframe,country_stats[['country_code','population']],
                                       on='country_code')
        
        dataframe=dataframe.set_index('country_code')
        
        print((dataframe['difference']/dataframe['population']).idxmax())

In [30]:
get_ratio(money_lent_borrowed_by_country)

NO


### 11) Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [31]:
get_ratio(money_lent_borrowed_by_country,True)

CA


### 12) For each year, compute the total amount of loans. 
Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. 

Per quest'ultimo punto si è operato nel seguente modo:<br>
*1)*: si è calcolato il valore assoluto della colonna *duration* del dataframe *loans*<br>
*2)*: si sono eliminati i valori nulli delle colonne *planned_expiration_time* e *disburse_time* del dataframe *loans*<br>
*3)*: per semplicità computazione si sono divisi tutti i prestiti con durata diversa da zero da quelli con durata zero

In [32]:
loans['duration'] = loans['duration'].abs()

loans_not_null = loans.dropna(subset = ['planned_expiration_time', 'disburse_time'])

loans__duration_not_zero = loans_not_null[loans_not_null['duration'] != 0]

loans_duration_zero = loans_not_null[loans_not_null['duration'] == 0]

La funzione **day_count** serve per calcolare il numero di giorni tra la data di inizio e di fine.<br>
La funzione restituisce una serie, creata a partire da un dizionario. Tale dizionario ha come chiave l'anno e il valore corrisponde la numero di giorni tra la data di inzio e di fine.<br>In particolare questo valore è dato dalla differenza tra il minimo valore tra la la data effettiva di fine e il primo gennaio dell'anno successivo e il massimo valore tra la data effettiva di inizio e il primo gennaio di quell'anno.<br>Questa funzione viene applicata solamente al dataframe di prestiti con durata diversa da zero.

In [33]:
def day_count(date1, date2):
    
    if date1 < date2:
        begin = date1 + datetime.timedelta(1)
        end = date2 + datetime.timedelta(1)
    else:
        begin = date2 + datetime.timedelta(1)
        end = date1 + datetime.timedelta(1)
        
    years = range(begin.year,end.year + 1)
    dictionary = dict()
    
    for year in years:
        year_start = datetime.date(year, 1, 1)
        year_end = datetime.date(year + 1, 1, 1)
        dictionary.update({year:(min(end, year_end) - max(begin, year_start)).days})
        
    return pd.Series(dictionary, name = 'Date_Value')

In [34]:
timetable = loans__duration_not_zero.apply(lambda x: day_count(x['disburse_time'], x['planned_expiration_time']), axis = 1)

timetable

,2011,2012,2013,2014,2015,2016,2017,2018
0,nan,nan,9.00000,45.00000,nan,nan,nan,nan
1,nan,nan,11.00000,85.00000,nan,nan,nan,nan
2,nan,nan,nan,37.00000,nan,nan,nan,nan
3,nan,nan,nan,35.00000,nan,nan,nan,nan
4,nan,nan,14.00000,44.00000,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...
1419602,nan,nan,nan,nan,38.00000,2.00000,nan,nan
1419603,nan,nan,nan,nan,37.00000,2.00000,nan,nan
1419604,nan,nan,nan,nan,48.00000,3.00000,nan,nan
1419605,nan,nan,nan,nan,58.00000,5.00000,nan,nan


I valori *NaN* del dataframe *timetable* (creato grazie alla funzione precedetemente descritta) vengono riempiti con zero; successivamente viene effettuato un merge tra questo dataframe e quello dei prestiti con durata diversa da zero, per poter ottenere informazioni sulla *duration* e sul *loan_amount*. 

In [35]:
timetable.fillna(0, inplace = True)

timetable = timetable.set_index(loans__duration_not_zero['loan_id']).reset_index()

timetable = pd.merge(timetable, loans__duration_not_zero[['loan_id', 'loan_amount', 'duration']], 
                     on = 'loan_id')

timetable

,loan_id,2011,2012,2013,2014,2015,2016,2017,2018,loan_amount,duration
0,657307,0.00000,0.00000,9.00000,45.00000,0.00000,0.00000,0.00000,0.00000,125.00000,54.00000
1,657259,0.00000,0.00000,11.00000,85.00000,0.00000,0.00000,0.00000,0.00000,400.00000,96.00000
2,658010,0.00000,0.00000,0.00000,37.00000,0.00000,0.00000,0.00000,0.00000,400.00000,37.00000
3,659347,0.00000,0.00000,0.00000,35.00000,0.00000,0.00000,0.00000,0.00000,625.00000,35.00000
4,656933,0.00000,0.00000,14.00000,44.00000,0.00000,0.00000,0.00000,0.00000,425.00000,58.00000
...,...,...,...,...,...,...,...,...,...,...,...
1044213,988180,0.00000,0.00000,0.00000,0.00000,38.00000,2.00000,0.00000,0.00000,400.00000,40.00000
1044214,988213,0.00000,0.00000,0.00000,0.00000,37.00000,2.00000,0.00000,0.00000,300.00000,39.00000
1044215,989109,0.00000,0.00000,0.00000,0.00000,48.00000,3.00000,0.00000,0.00000,2425.00000,51.00000
1044216,989143,0.00000,0.00000,0.00000,0.00000,58.00000,5.00000,0.00000,0.00000,100.00000,63.00000


Viene di seguito creato un nuovo dataframe, chiamato *loan_amount_per_year*, in cui viene calcolato in modo proporzionale l'ammontare dei prestiti per i singoli anni.

In [36]:
loan_amount_per_year = timetable[[2011,2012,2013,2014,2015,2016,2017,2018
                        ]].div(timetable['duration'], axis=0).mul(timetable['loan_amount'], axis = 0)

Viene calcolata la somma dei prestiti anno per anno sul dataframe appena creato (*df1*) e anche la somma dei prestiti anno per anno sul dataframe dei prestiti con durata uguale a zero (*df2*).<br>Infine questi due dataset vengono concatenati, viene eseguita la somma tra i *loan_amount* dei dataframe *df1* e *df2*, anno per anno, ed infine viene stampato il dataframe finale, chiamato *loan_amount_per_year_complete*.  

In [37]:
df1 = loan_amount_per_year.sum().to_frame().reset_index().rename(columns = {'index':'year', 0:'loan_amount'})

df2 = loans_duration_zero.groupby('disburse_year')['loan_amount'].sum().to_frame().reset_index().\
       rename(columns = {'disburse_year':'year'})

In [38]:
loan_amount_per_year_complete = pd.concat([df1,df2]).groupby('year')['loan_amount'].sum().to_frame().reset_index()

In [39]:
loan_amount_per_year_complete

,year,loan_amount
0,2011.00000,629911.77488
1,2012.00000,113287580.79478
2,2013.00000,125015552.04954
3,2014.00000,153214272.06143
4,2015.00000,159929916.50857
5,2016.00000,159363790.44077
6,2017.00000,172536423.51457
7,2018.00000,5826652.85546
